###### imports

In [2]:
import pandas as pd # for dataframe analysis
import numpy as np # for arrays functionality, i.e. .where()...
import re
from functools import partialmethod # for changing default of pandas .head()

###### attributions

[1) Set default .head() to 3 rows -- Ted Petrou, Dunder Data](https://medium.com/dunder-data/pandas-trick-1-change-the-default-number-of-rows-returned-from-the-head-method-bc7c21ce0d53)

###### set defaults

In [3]:
# attribution 1
pd.DataFrame.head = partialmethod(pd.DataFrame.head, n=3)

###### data cleaning

In [4]:
whfoods = pd.read_csv('./Data/whfoods.csv')

In [5]:
whfoods.index = range(whfoods.shape[0])

In [6]:
whfoods

,"Asparagus, Cooked",Unnamed: 1,Unnamed: 2,"Avocado, cubed, raw",Unnamed: 4,Unnamed: 5,"Beet Greens, boiled",Unnamed: 7,Unnamed: 8,"Beets, sliced, cooked",...,Unnamed: 341,"Sage, dried",Unnamed: 343,Unnamed: 344,"Thyme, fresh",Unnamed: 346,Unnamed: 347,"Turmeric, ground",Unnamed: 349,Unnamed: 350
0,BASIC MACRONUTRIENTS AND CALORIES,NaN,NaN,BASIC MACRONUTRIENTS AND CALORIES,NaN,NaN,BASIC MACRONUTRIENTS AND CALORIES,NaN,NaN,BASIC MACRONUTRIENTS AND CALORIES,...,NaN,BASIC MACRONUTRIENTS AND CALORIES,NaN,NaN,BASIC MACRONUTRIENTS AND CALORIES,NaN,NaN,BASIC MACRONUTRIENTS AND CALORIES,NaN,NaN
1,nutrient,amount,DRI/DV,nutrient,amount,DRI/DV,nutrient,amount,DRI/DV,nutrient,...,DRI/DV,nutrient,amount,DRI/DV,nutrient,amount,DRI/DV,nutrient,amount,DRI/DV
2,NaN,NaN,(%),NaN,NaN,(%),NaN,NaN,(%),NaN,...,(%),NaN,NaN,(%),NaN,NaN,(%),NaN,NaN,(%)
3,Protein,4.32 g,9,Protein,3.00 g,6,Protein,3.70 g,7,Protein,...,0,Protein,0.15 g,0,Protein,0.27 g,1,Protein,0.34 g,1
4,Carbohydrates,7.40 g,3,Carbohydrates,12.80 g,6,Carbohydrates,7.86 g,3,Carbohydrates,...,0,Carbohydrates,0.85 g,0,Carbohydrates,1.17 g,1,Carbohydrates,2.86 g,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Artificial Sweeteners (Total),-- mg,NaN,Artificial Sweeteners (Total),-- mg,NaN,Artificial Sweeteners (Total),-- mg,NaN,Artificial Sweeteners (Total),...,NaN,Artificial Sweeteners (Total),-- mg,NaN,Artificial Sweeteners (Total),-- mg,NaN,Artificial Sweeteners (Total),-- mg,NaN
170,Aspartame,-- mg,NaN,Aspartame,-- mg,NaN,Aspartame,-- mg,NaN,Aspartame,...,NaN,Aspartame,-- mg,NaN,Aspartame,-- mg,NaN,Aspartame,-- mg,NaN
171,Saccharin,-- mg,NaN,Saccharin,-- mg,NaN,Saccharin,-- mg,NaN,Saccharin,...,NaN,Saccharin,-- mg,NaN,Saccharin,-- mg,NaN,Saccharin,-- mg,NaN
172,Alcohol,0.00 g,NaN,Alcohol,0.00 g,NaN,Alcohol,0.00 g,NaN,Alcohol,...,NaN,Alcohol,0.00 g,NaN,Alcohol,0.00 g,NaN,Alcohol,0.00 g,NaN


In [7]:
# investigating dimensions of dataset

nfoods = len(whfoods.columns)/3 # there are 117 foods in the dataset.
nfoods
nnutrients = whfoods.shape[0]
whfoods.shape

(174, 351)

**Initial assessment of dataset**\
\
*SHAPE:* There are three columns per food group, 177 rows, 1000 columns.  \
\
*INFORMATION:* There is no serving size in grams of the food, though from inspecting the excel spreadsheet it appears possible to reconstruct this value from the ammounts of the macronutrients, micronutrients, water, and ash.\
\
*FORMATTING:* The three columns indicate first the description of the component of the food, the amount in grams, milligrams, or micrograms, and the DRI/DV if applicable.

**Action Items**
1) Generate two dataframes -- one for amount, one for DRI/DV
2) Standardize missing value indicators
3) Eliminate rows with one unique value.
4) Separate out units for the amount df into a reference dictionary of {nutrient1: 'mg', nutrient2: 'g', nutrient3: 'mcg', ...}
5) Build the LP problem with PuLP

In [8]:
# initial pass on removing extraneous rows, columns.
whfoods = whfoods.dropna(how='all',axis=1).dropna(how='all', axis=0)

There are under 173 nutrient categories since some of the rows correspond to supercategories such as 'Minerals','INDIVIDUAL FATTY ACIDS', 'Monounsaturated Fats', 'INDIVIDUAL AMINO ACIDS', 'OTHER COMPONENTS', etc. as well as corresponding extraneous rows such as: 'nutrient', nan.

The dataset could be simplified by removing rows with all 0.00 g, mg etc or all -- mgsuch as with the sweeteners, caffeine, alcohol.

I'll create two dictionaries, one of the foods and the raw nutrient values and another of the foods and DRI/DV.  These can be used to generate data frames.  The raw nutrient values and the DRI/DV info can be used to generate the 100% DRI/DV values for each nutrient.  Rounding errors can be overcome by using the food with the highest DRI/DV for a given nutrient to generate the recommendation.

###### column name formatting

In [9]:
# gather all the foods into a list
foods = [food for inx,food in enumerate(whfoods.columns) if inx%3 ==0]
len(foods) # verify the number of foods

117

In [10]:
# 
new_cols = []
for i, food in enumerate(foods):
    # Replace spaces with a single underscore
    food = re.sub(r' +', '_', food)
    # Replace commas with a single underscore
    food = re.sub(r',+', '_', food)
    # Remove duplicate underscores
    food = re.sub(r'_+', '_', food)
    new_cols = new_cols + [food, f'NV_{food}', f'DRVs_{food}']

In [11]:
whfoods.columns = new_cols

In [12]:
whfoods.head()

,Asparagus_Cooked,NV_Asparagus_Cooked,DRVs_Asparagus_Cooked,Avocado_cubed_raw,NV_Avocado_cubed_raw,DRVs_Avocado_cubed_raw,Beet_Greens_boiled,NV_Beet_Greens_boiled,DRVs_Beet_Greens_boiled,Beets_sliced_cooked,...,DRVs_Rosemary_fresh,Sage_dried,NV_Sage_dried,DRVs_Sage_dried,Thyme_fresh,NV_Thyme_fresh,DRVs_Thyme_fresh,Turmeric_ground,NV_Turmeric_ground,DRVs_Turmeric_ground
0,BASIC MACRONUTRIENTS AND CALORIES,NaN,NaN,BASIC MACRONUTRIENTS AND CALORIES,NaN,NaN,BASIC MACRONUTRIENTS AND CALORIES,NaN,NaN,BASIC MACRONUTRIENTS AND CALORIES,...,NaN,BASIC MACRONUTRIENTS AND CALORIES,NaN,NaN,BASIC MACRONUTRIENTS AND CALORIES,NaN,NaN,BASIC MACRONUTRIENTS AND CALORIES,NaN,NaN
1,nutrient,amount,DRI/DV,nutrient,amount,DRI/DV,nutrient,amount,DRI/DV,nutrient,...,DRI/DV,nutrient,amount,DRI/DV,nutrient,amount,DRI/DV,nutrient,amount,DRI/DV
2,NaN,NaN,(%),NaN,NaN,(%),NaN,NaN,(%),NaN,...,(%),NaN,NaN,(%),NaN,NaN,(%),NaN,NaN,(%)


In [13]:
# gather reformatted food names into a list
foods = [food for inx,food in enumerate(whfoods.columns) if inx%3 == 0]
len(foods) # verify the number of foods

117

In [14]:
nnutrients

174

Folate (DFE), Folate, and Folate (food) on first glance have the same values.  If this is the case I'll only keep Folate.   

###### separate nutrient val, drv info

In [15]:
# collect a nested dictionary of... {food: {nutrient:nutrient_val}}
nutrient_vals = \
{food:
    {whfoods.loc[i,food]: 
     whfoods.iloc[i,int(np.where(whfoods.columns.values==food)[0][0])+1] 
     for i in range (2, nnutrients)
    } for food in foods
}

# collect a nested dictionary of... {food: {nutrient:nutrient_drv}}
nutrient_drv = \
{food:
    {whfoods.loc[i,food]: 
     whfoods.iloc[i,int(np.where(whfoods.columns.values==food)[0][0])+2] 
     for i in range (2, nnutrients)
    } for food in foods
}

# convert to DatFrame
nutrient_vals_df = pd.DataFrame(nutrient_vals)
nutrient_drv_df = pd.DataFrame(nutrient_drv)

# Transposing so that the foods are like "observations" in long format
nutrient_vals_df = nutrient_vals_df.T
nutrient_drv_df = nutrient_drv_df.T

In [16]:
nutrient_vals_df.columns.values

array([nan, 'Protein', 'Carbohydrates', 'Fat - total', 'Dietary Fiber',
       'Calories', 'MACRONUTRIENT AND CALORIE DETAIL', 'nutrient',
       'Carbohydrate:', 'Starch', 'Total Sugars', 'Monosaccharides',
       'Fructose', 'Glucose', 'Galactose', 'Disaccharides', 'Lactose',
       'Maltose', 'Sucrose', 'Soluble Fiber', 'Insoluble Fiber',
       'Other Carbohydrates', 'Fat:', 'Monounsaturated Fat',
       'Polyunsaturated Fat', 'Saturated Fat', 'Trans Fat',
       'Calories from Fat', 'Calories from Saturated Fat',
       'Calories from Trans Fat', 'Cholesterol', 'Water',
       'MICRONUTRIENTS', 'Vitamins', 'Water-Soluble Vitamins',
       'B-Complex Vitamins', 'Vitamin B1', 'Vitamin B2', 'Vitamin B3',
       'Vitamin B3 (Niacin Equivalents)', 'Vitamin B6', 'Vitamin B12',
       'Biotin', 'Choline', 'Folate', 'Folate (DFE)', 'Folate (food)',
       'Pantothenic Acid', 'Vitamin C', 'Fat-Soluble Vitamins',
       'Vitamin A (Retinoids and Carotenoids)',
       'Vitamin A Internationa

###### Cleaning

Remove all columns that have uniform values

In [17]:
# credit: ChatGPT
def remove_columns_with_same_value(df):
    # Calculate the number of unique values in each column
    unique_counts = df.nunique()

    # Get the column names where the number of unique values is 1
    columns_to_remove = unique_counts[(unique_counts == 1)|(unique_counts == 0)].index

    # Remove the columns from the dataframe
    df = df.drop(columns=columns_to_remove)

    return df

In [18]:
nutrient_vals_df = \
nutrient_vals_df.drop(
    columns=['nutrient', 'MACRONUTRIENT AND CALORIE DETAIL',np.nan])

In [19]:
nutrient_vals_df = remove_columns_with_same_value(nutrient_vals_df)

In [59]:
nutrient_vals_df.head(n=1)

,Protein,Carbohydrates,Fat - total,Dietary Fiber,Calories,Starch,Total Sugars,Monosaccharides,Fructose,Glucose,...,Tryptophan,Tyrosine,Valine,Ash,Organic Acids (Total),Acetic Acid,Citric Acid,Malic Acid,Sugar Alcohols (Total),Xylitol
Asparagus_Cooked,4.32 g,7.40 g,0.40 g,3.60 g,39.6,0.00 g,2.34 g,2.18 g,1.42 g,0.76 g,...,0.05 g,0.10 g,0.22 g,1.13 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g


Verify whether the three folate forms of 'Folate', 'Folate (DFE)', and 'Folate (food)' are the same for all foods.

In [71]:
len(nutrient_vals_df.index.values)

117

In [72]:
nutrient_vals_df[['Folate', 'Folate (DFE)', 'Folate (food)']].apply(lambda x: x.nunique() == 1,axis = 1).sum()

116

One food does not have the same values for each folate category. 

In [75]:
nutrient_vals_df[nutrient_vals_df[['Folate', 'Folate (DFE)', 'Folate (food)']].\
                 apply(lambda x: x.nunique() != 1,axis = 1).values][[
    'Folate', 'Folate (DFE)', 'Folate (food)']]

,Folate,Folate (DFE),Folate (food)
Sea_Vegetables_dulse_dried,0.00 mcg,45.60 mcg,0.00 mcg


In [21]:
for col in nutrient_vals_df.columns:
    nutrient_vals_df[col] = nutrient_vals_df[col].str.replace('--','0.00')

In [22]:
nutrient_vals_df

,Protein,Carbohydrates,Fat - total,Dietary Fiber,Calories,Starch,Total Sugars,Monosaccharides,Fructose,Glucose,...,Sugar Alcohols (Total),Glycerol,Inositol,Mannitol,Sorbitol,Xylitol,Artificial Sweeteners (Total),Aspartame,Saccharin,Caffeine
Asparagus_Cooked,4.32 g,7.40 g,0.40 g,3.60 g,39.6,0.00 g,2.34 g,2.18 g,1.42 g,0.76 g,...,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 mg,0.00 mg,0.00 mg,0.00 mg
Avocado_cubed_raw,3.00 g,12.80 g,21.99 g,10.05 g,240,0.00 g,0.99 g,0.89 g,0.18 g,0.56 g,...,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 mg,0.00 mg,0.00 mg,0.00 mg
Beet_Greens_boiled,3.70 g,7.86 g,0.29 g,4.18 g,38.88,0.00 g,0.86 g,0.00 g,0.00 g,0.00 g,...,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 mg,0.00 mg,0.00 mg,0.00 mg
Beets_sliced_cooked,2.86 g,16.93 g,0.31 g,3.40 g,74.8,0.00 g,13.53 g,0.00 g,0.00 g,0.00 g,...,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 mg,0.00 mg,0.00 mg,0.00 mg
Bell_Peppers_sliced_red_raw,0.91 g,5.55 g,0.28 g,1.85 g,28.52,0.00 g,3.70 g,3.70 g,1.99 g,1.71 g,...,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 mg,0.00 mg,0.00 mg,0.00 mg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Peppermint_leaves_fresh,0.28 g,1.13 g,0.07 g,0.61 g,5.32,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,...,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 mg,0.00 mg,0.00 mg,0.00 mg
Rosemary_fresh,0.14 g,0.87 g,0.25 g,0.59 g,5.5,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,...,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 mg,0.00 mg,0.00 mg,0.00 mg
Sage_dried,0.15 g,0.85 g,0.18 g,0.56 g,4.41,0.00 g,0.02 g,0.00 g,0.00 g,0.00 g,...,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 mg,0.00 mg,0.00 mg,0.00 mg
Thyme_fresh,0.27 g,1.17 g,0.08 g,0.67 g,4.85,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,...,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 mg,0.00 mg,0.00 mg,0.00 mg


In [23]:
nutrient_vals_df = remove_columns_with_same_value(nutrient_vals_df)

In [24]:
nutrient_vals_df

,Protein,Carbohydrates,Fat - total,Dietary Fiber,Calories,Starch,Total Sugars,Monosaccharides,Fructose,Glucose,...,Tryptophan,Tyrosine,Valine,Ash,Organic Acids (Total),Acetic Acid,Citric Acid,Malic Acid,Sugar Alcohols (Total),Xylitol
Asparagus_Cooked,4.32 g,7.40 g,0.40 g,3.60 g,39.6,0.00 g,2.34 g,2.18 g,1.42 g,0.76 g,...,0.05 g,0.10 g,0.22 g,1.13 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g
Avocado_cubed_raw,3.00 g,12.80 g,21.99 g,10.05 g,240,0.00 g,0.99 g,0.89 g,0.18 g,0.56 g,...,0.04 g,0.07 g,0.16 g,2.37 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g
Beet_Greens_boiled,3.70 g,7.86 g,0.29 g,4.18 g,38.88,0.00 g,0.86 g,0.00 g,0.00 g,0.00 g,...,0.06 g,0.09 g,0.11 g,3.80 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g
Beets_sliced_cooked,2.86 g,16.93 g,0.31 g,3.40 g,74.8,0.00 g,13.53 g,0.00 g,0.00 g,0.00 g,...,0.03 g,0.07 g,0.10 g,1.90 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g
Bell_Peppers_sliced_red_raw,0.91 g,5.55 g,0.28 g,1.85 g,28.52,0.00 g,3.70 g,3.70 g,1.99 g,1.71 g,...,0.01 g,0.01 g,0.03 g,0.43 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Peppermint_leaves_fresh,0.28 g,1.13 g,0.07 g,0.61 g,5.32,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,...,0.00 g,0.01 g,0.01 g,0.13 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g
Rosemary_fresh,0.14 g,0.87 g,0.25 g,0.59 g,5.5,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,...,0.00 g,0.00 g,0.01 g,0.10 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g
Sage_dried,0.15 g,0.85 g,0.18 g,0.56 g,4.41,0.00 g,0.02 g,0.00 g,0.00 g,0.00 g,...,0.00 g,0.00 g,0.00 g,0.11 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g
Thyme_fresh,0.27 g,1.17 g,0.08 g,0.67 g,4.85,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,...,0.01 g,0.00 g,0.01 g,0.15 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g,0.00 g


##### Strip units, create {nutrient: unit} dictionary

The whfoods data contain the following units:\
mg (ATE), mcg (RE), mcg (RAE), IU, g, mg, mcg

In [56]:
# Generate a dictionary of units for the different info types (serving_size, 
# calories, etc.)

units_dict = {}
for r in range(1,len(nutrient_vals_df.columns)):  # for each of the nutrient labels
    pattern_units = re.compile(r"[mcgiu(RAE)T\)\(]+",re.I) # generate pattern object
    i = 0 # start with 1st nutrient value
    s = nutrient_vals_df.iat[i,r] # get nutrient value
    m = pattern_units.search(s) # check pattern object against nutrient value, 
                        # generating match object
    # try:
    #     print(m.group())
    # except:
    #     print("no match")
    try:
        units[nutrient_vals_df.columns[r]] = m.group()
    except:
        while not m:
            # loop through until we have a unit or decide there are no units
            i = i+1 # go to next value
            s = nutrient_vals_df.iat[i,r] # store value
            m = pattern_units.search(s) # search value against the pattern

            try: # store match if there is one
                units[nutrient_vals_df.columns[r]] = m.group() 
            except: 
                if i==nutrient_vals_df.shape[0]-1: # if no units found by end, None units
                    m = "None"
                    units_dict[nutrient_vals_df.columns[r]] = m

With units stored in units_dict, I can convert the info to 

In [30]:
nutrient_vals_df['Acetic Acid'].value_counts()

0.00 g    116
0.01 g      1
Name: Acetic Acid, dtype: int64

In [ ]:
nutrient_vals_df[nutrient_vals_df['Saccharin']=='#NAME?']

In [ ]:
# Select rows with '#NAME' in any column
mask = nutrient_vals_df.apply(lambda x: x.str.contains('#NAME?')).any(axis=1)
rows_with_name = nutrient_vals_df[mask]

In [ ]:
rows_with_name.index.values[0]

In [ ]:
rows_with_name.T['Cheese_grass_fed_cheddar_whole_milk']

In [ ]:
nutrient_vals_df.loc['Cheese_grass_fed_cheddar_whole_milk',:]

In [ ]:
nutrient_vals_df.loc['Cheese_grass_fed_cheddar_whole_milk',:]

In [ ]:
nutrient_vals_df.loc[['Cheese_grass_fed_cheddar_whole_milk']] = \
rows_with_name.T[rows_with_name.index.values[0]].str.replace(
    '#NAME?','0.00',regex=False).str.replace('?',"",regex=False).to_frame().T


In [ ]:
nutrient_vals_df

In [ ]:
rows_with_name.where.replace()

In [ ]:
nutrient_vals_df.query()

In [ ]:
nutrient_vals_df['15:1 Pentadecenoic'].str.replace("--","0.00")

In [ ]:
nutrient_vals_df.query("Aspartame !=")

In [ ]:
nutrient_vals_df.nunique().sort_values()[0:20]

In [ ]:
conversions = 0
for nutrient in nutrient_drv_df.columns.values[[0,1]]:
    try: 
        nutrient_drv_df[[nutrient]] = nutrient_drv_df[[nutrient]].astype("int")
        conversions += 1
    except: pass

In [ ]:
conversions

In [ ]:
nutrient_drv_df

In [ ]:
nutrient_drv_df.nunique()[10:40]

To simplify & clean the dataset, I'll collect the "nutrient_drv" columns that don't carry any information, i.e. have the same values for all the foods.

In [ ]:
[nutrient_drv_df[]for ]